In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import scipy
from scipy.spatial.transform import Rotation as R

In [ ]:
cart_df = pd.read_csv("..//wheel_cart_teensy.csv")

In [ ]:
cart_df

In [ ]:
mc = pd.read_csv("..//mocap_data.csv", skiprows=6)

In [ ]:
st_time_mc = "2022-06-09 06.15.20.583 PM"
mc_dt = datetime.strptime(st_time_mc, "%Y-%m-%d %I.%M.%S.%f %p")
mc_dt

In [ ]:
mc = mc.rename(columns={"Time (Seconds)":"seconds"})

In [ ]:
_t = []
for i in list(mc["seconds"]):
    _t.append(mc_dt + timedelta(0,i))

mc["time"] = _t

In [ ]:
mc

In [ ]:
"""getting initial values of motion capture data"""

marker_cen = np.array(mc[["center_x", "center_y", "center_z"]].iloc[0]).T
marker_xvec = np.array(mc[["xvec_x", "xvec_y", "xvec_z"]].iloc[0]).T
marker_zvec = np.array(mc[["zvec_x", "zvec_y", "zvec_z"]].iloc[0]).T
marker_org = np.array(mc[["org_x", "org_y", "org_z"]].iloc[0]).T

In [ ]:
v1 = marker_xvec - marker_org #v1
v2 = marker_zvec - marker_org #v2

v1 = v1.reshape(3,1)
v2 = v2.reshape(3,1)
v3 = marker_org.reshape(3,1)

In [ ]:
def calculate_rotmat(xdir,zdir,org):
    """
    this function calculates rotation matrix
    """
    v1 = xdir - org #v1
    v2 = zdir - org #v2

    vxnorm = v1/np.linalg.norm(v1)

    vzcap = v2 - (vxnorm.T @ v2) * vxnorm
    vznorm = vzcap/ np.linalg.norm(vzcap)

    vynorm = np.cross(vznorm.T[0], vxnorm.T[0]).reshape(3,1)
    rotMat = np.hstack((vxnorm, vynorm, vznorm))
    return rotMat

In [ ]:
"""getting initial rot mat in mc data"""

rot_mat = calculate_rotmat(v1, v2, v3)
rot_mat

In [ ]:
temp_list = []
for i in range(len(mc["xvec_x"])):
    center_val =  np.array(mc[["center_x", "center_y", "center_z"]].iloc[i])
    center_val = center_val.reshape(3,1)
    transformed_center = rot_mat.T @ center_val + v3
    transformed_center = transformed_center.T[0]
    temp_list.append(transformed_center)
mc[["cen_x", "cen_y", "cen_z"]] = temp_list

In [ ]:
mc

In [ ]:
plt.plot(mc["cen_x"])

In [ ]:
"""resetting cart values to zero"""
cart_df["e_fr"] = cart_df["e_fr"]- cart_df["e_fr"].iloc[0]
cart_df["e_fl"] = cart_df["e_fl"]- cart_df["e_fl"].iloc[0]
cart_df["e_rr"] = cart_df["e_rr"]- cart_df["e_rr"].iloc[0]
cart_df["e_rl"] = cart_df["e_rl"]- cart_df["e_rl"].iloc[0]

cart_df

In [ ]:
mils = np.arange(0, len(cart_df["e_fr"])*10, 10)
cart_df["mils"] = mils

These are the parameters of the cart

Diameter = 95 mm
radius = 47.5
wheel thickness = 45 mm
gap between wheel and chassis = 6.5 mm
angle between center of chassis and z-dir vector = 51.21 degrees
distance between the wheel = 158 mm
distance between the wheel and the center of the robot = 101.36 (li)


In [ ]:
"""circumfrence of the wheel"""

cir_wheel = 2*np.pi*47.5
cir_wheel
radius = 47.5

lx = 79 #half of the distance between the wheels
ly = 122.5/2

The encoder values will give 4000 values per revolution
360/4000 = 0.09 degrees per encoder rotation value

In [ ]:
""" angular velocity"""
cart_df["av_fr"] = (cart_df["e_fr"]*0.09).diff()/10
cart_df["av_fl"] = (cart_df["e_fl"]*0.09).diff()/10
cart_df["av_rr"] = (cart_df["e_rr"]*0.09).diff()/10
cart_df["av_rl"] = (cart_df["e_rl"]*0.09).diff()/10
cart_df['av_fr'] = cart_df["av_fr"].fillna(0)
cart_df['av_fl'] = cart_df["av_fl"].fillna(0)
cart_df['av_rr'] = cart_df["av_rr"].fillna(0)
cart_df['av_rl'] = cart_df["av_rl"].fillna(0)
cart_df["av_fl"]

In [ ]:
"""finding vx, vy, w"""

cart_df["vx"] = (cart_df["av_fr"] + cart_df["av_fl"] + cart_df["av_rl"] + cart_df["av_rr"])*(radius/4)
cart_df["vy"] = (- cart_df["av_fr"] + cart_df["av_fl"] + cart_df["av_rl"] - cart_df["av_rr"])*(radius/4)
cart_df["w"] = (- cart_df["av_fr"] + cart_df["av_fl"] - cart_df["av_rl"] + cart_df["av_rr"])*(radius/(4*(lx + ly)))

In [ ]:
cart_df["vx"]

In [ ]:
"""calculating displacement
s=(1/2)* (v+u)t
v = current velocity
u = initial velocity
t = time
s = displacement
"""
_xval = []
_yval = []
for i in range(len(cart_df["vx"])):
    _xval.append(0.5*(cart_df["vx"].iloc[i] + cart_df["vx"].iloc[0])*10)
    _yval.append(0.5*(cart_df["vy"].iloc[i] + cart_df["vy"].iloc[0])*10)
cart_df["x_val"] = _xval
cart_df["y_val"] = _yval

In [ ]:
plt.plot(cart_df["x_val"])
plt.plot(cart_df["y_val"])

In [ ]:
plt.plot(mc["cen_z"])